# Long Island Jewish Medical Center | Northwell Health—Energy Data Analysis Project
### Created by Evan McFarland

HOW TO READ THIS PROGRAM: Refer to README file for project scope. This program uses a very simple chronological setup. Each section is a separated concept. Each subsection has 3 blocks that are titled with "variables, values, and equations" which should leave no doubt as to what each variable means, where it comes from, and how it is used. Variables are not repeated so if sections in the lower parts of this program do not redefine a variable, that means it was already defined in previous sections.

Each value here is on an annual baisis but the code has no time dimension so units do not specify time frames. This program will run for daily, weekly, or monthly data so long as all of the variables are consistent with that time frame. Every value that does not have a source comes from plant data.

## Section 1: Using Conservation of Energy to Calculate Efficiency

### Section 1.1: Conversion Factors

- There a few nuances of the plant that need to be accounted for before diving into calculations. Two correction factors are calculated here and used throughout the program:
    - Therms in boilers - Converts the theoretical therms that are released from perfect combustion of liquid natral gas. This is nessasary because cogen units given are already in therms while boiler is not.
    - Boiler maintenence correction factor - Both boilers and cogeneration accrrue maintence costs but they output different amounts of energy. Since the boilers output more than 5 times the energy of cogen, their maintenence costs must be equalized to account for this. 

#### Conversion Factor Variables

In [22]:
therms_in_boilers = 'tot therms (converted from natral gas) that go into the boiler'
gas_to_therms = 'conversion factor for energy in therms per cubic foot of natral gas'   # retreived from https://sciencing.com/convert-cubic-feet-therms-8374234.html
gas_in_boilers = 'how much gas is actually put into the boilers in ft^3' 
therms_in_cogen = 'how many therms enter into the cogen plant (given by sponsor)'
boiler_maintence_correction_factor = 'this accounts for the fact that the boiler has greater energy output and its maintence costs get weighted' 

#### Conversion Factor Values

In [23]:
gas_in_boilers = 443670000
gas_to_therms = 0.01034       # constant from source above
therms_in_cogen = 751635

#### Conversion Factor Equations

In [24]:
therms_in_boilers = gas_in_boilers * gas_to_therms
boiler_maintence_correction_factor = therms_in_boilers / therms_in_cogen
therms_in_boilers = round(therms_in_boilers, 4)
boiler_maintence_correction_factor = round(boiler_maintence_correction_factor, 2)
print('Therms Into Boilers')
print(therms_in_boilers)
print()
print('Boiler Maintenence Correction Factor')
print(boiler_maintence_correction_factor)

Therms Into Boilers
4587547.8

Boiler Maintenence Correction Factor
6.1


### Section 1.2: Cogeneration

#### Cogeneration Variables

In [25]:
Nc = 'Efficency of CoGen'
Nc_econ = 'Economic Efficency of Cogen' 
therms_out_cogen = 'how much heat was recovered from cogen exaust gasses'

#### Cogeneration Values

In [26]:
therms_out_cogen = 339348
ratio_kwh_to_therms = 0.034130    # retreived from https://www.metric-conversions.org/energy-and-power/kilowatt-hours-to-therms-us.htm
kwh_generated = 7710200

#### Cogeneration Equations

In [27]:
kwh_therms_equivalent = kwh_generated * ratio_kwh_to_therms
Nc = (therms_out_cogen + kwh_therms_equivalent) / therms_in_cogen

### Section 1.3: Boilers

#### Boiler Variables

In [28]:
S_mass = 'steam output lbm/month'
S_energy = 'steam btu/lbm out of boilers #from steam tables'  # retreived from https://www.engineeringtoolbox.com/saturated-steam-properties-d_273.html
S_total_energy = 'steam total monthly energy'    
G_energy_lbm = 'gas energy (btu/lbm)'  # retreived from https://www.engineeringtoolbox.com/heating-values-fuel-gases-d_823.html
G_vol = 'gas input to boilers (cubic feet)'   # given by sponser
G_energy_ft3 = 'gas energy  (btu/cubic foot)'  # retreived from https://www.engineeringtoolbox.com/heating-values-fuel-gases-d_823.html
Nb = 'boiler efficiency'
Nb_econ = 'boiler economic efficiency'

#### Boiler Values

In [29]:
S_mass = 367597000
stm_energy = 1193         # constant - this is enthalpy in btu/lb satturated vapor at 120 psi from source above
S_energy = 1013           # This is steam out energy - deairator in energy (180btu/lbm) which is the properties of water in a 212F deairator
G_vol = gas_in_boilers
G_energy_ft3 = 1034     # constant - 1034btu/ft^3 of natral gas and consistent with 1therm/99976btu, i.e., the value for "gas_to_therms"
G_vol_to_therm = 96.7   # in ft^3 and gives you therms when dividing 96.7 ft^3 

#### Boiler Calculations

In [30]:
G_energy_btu = G_vol * G_energy_ft3       #(G_vol_to_therm / gas_to_therms)
S_total_energy = S_mass * S_energy        #G_total_energy = G_energy_btu * G_energy_ft3  (therms_out_cogen * 99976)
Nb = (S_total_energy) / G_energy_btu 
print(S_total_energy)

372375761000


In [31]:
print('Cogeneration Plant Efficiency')
Nc = round(Nc, 4)
print(Nc * 100,'%')
print('\n')
print('Boiler Plant Efficiency')
Nb = round(Nb, 4)
print(Nb * 100,'%')

Cogeneration Plant Efficiency
80.16 %


Boiler Plant Efficiency
81.17 %


## Section 2: Calculating Economic Efficiency (Actual Efficiency)

## Economic Efficiency
Sections 1-3 used conservation of energy to determine theoretical efficiency of boilers and cogeneration. The results clearly favor boilers in terms of efficiency but this does  not provide a comprehensive picture of plant efficiency. 
    
Local laws 95 and 97 track consumption per square foot when rating the sustainability of buildings. Thus, legislative consideration of other energy production factors comes to a halt at this point in the analysis. The remainder of this program performs comprehensive analysis of building sustainability by adding previously ignored factors—namely, the realization that on site electricity generation reduces regional carbon emmissions. 

Economic Efficiency is a hypothetical ratio used to represent actual dollar value efficiency, i.e., (dollars_in / dollars_out) instead of (Energy_in / Energy_out). This is a better representation of efficiency when comparing cogeneration and boiler output side by side because they don't produce the same type of energy. For example and as we'll see, the theoretical energy equivalents of kWh and Therms for a powerplant distributing energy to NYC is actually 4.5 times more valuable in kWh form than in Therm form. This is because converting fuel into heat is much more energy efficient than converting fuel into electricity. From the perspective of Local Laws 95 and 97, this is perhaps easy to overlook because it is designed for buildings, not hospitals (which uniquely need on site electricity generation).

### Section 2.1: Cogeneration 

#### Cogeneration Variables

In [32]:
kwh_generated = 'kwh generated per month (kWh)'
cogen_maintenence_therm = 'maintence cost converted to therms'

#### Cogeneration Values

In [33]:
cogen_maintence_therms = 369226
dollars_per_therm_interuptable = 0.45       # constant - from LIJ energy bills
dollars_per_therm_noninteruptable = 0.60    # constant - from LIJ energy bills (not currently used but this is subject to change)
dollars_per_kwh = 0.115                     # constant - from LIJ energy bills
kwh_generated = 7710200

#### Cogeneration Equations

In [34]:
savings_kwh = kwh_generated * dollars_per_kwh
savings_therms = therms_out_cogen * dollars_per_therm_interuptable
cost_therms = therms_in_cogen * dollars_per_therm_noninteruptable
cost_maintence = cogen_maintence_therms * dollars_per_therm_noninteruptable
Nc_econ = (savings_kwh + savings_therms) / (cost_therms + cost_maintence)

### Section 2.2: Boilers

#### Boiler Variables

In [35]:
boiler_maintence_cost = 'how much was spent on boiler maintence including material, labor, time, and operation costs' # given by sponsor 
gas_dollar_value = 'the cost of the gas that is burned in the boilers (dollars)'
Nb_econ =  'final economic efficiency of boilers'
total_boiler_maintence_costs = 'total maintence costs after adding correction factor'
boiler_final_cost = 'total cost for the plant to run boilers at current rates of operation'

#### Boiler Values

In [36]:
boiler_maintence_cost = 10000.0

#### Boiler Equations

In [37]:
gas_dollar_value = (gas_to_therms * G_vol) * dollars_per_therm_interuptable
total_boiler_maintence_costs = boiler_maintence_cost * boiler_maintence_correction_factor
boiler_energy_out_dollar_value = (Nb * gas_dollar_value) 
boiler_final_cost = gas_dollar_value + total_boiler_maintence_costs
Nb_econ = boiler_energy_out_dollar_value / boiler_final_cost 

## Results

In [38]:
print('Final Answers:')
print('\n')
print('Cogeneration Economy Ratio')
Nc_econ = round(Nc_econ, 4)
print(Nc_econ) 
print()
print('Boiler Economy Ratio')
Nb_econ = round(Nb_econ, 4)
print(Nb_econ)
print()
print('Generator Economy Ratio')
generator_economic_efficiency = savings_kwh / (kwh_therms_equivalent * dollars_per_therm_noninteruptable)   
# This equation which shows how much more ecomomically efficient kWh are than therms, i.e., we assumed 
# theoretical conversion rates between kWh and therms but in reality kWh are 4 to 5 times more valuble
# than therms. This is because energy is always lost in the process of converting heat to electricity.
# This also explains why Cogen has less raw efficiency (Wout/Win) but higher economic efficiency.
generator_economic_efficiency = round(generator_economic_efficiency, 3)
print(generator_economic_efficiency)

Final Answers:


Cogeneration Economy Ratio
1.5455

Boiler Economy Ratio
0.7884

Generator Economy Ratio
5.616


##### Explanation of Results
   Cogeneration Economic Efficiency - when accounting for the economic value of producing on site electricity and utilization of waste heat, cogeneration nearly doubles the economic value that is put into it. When accounting for a generous plant efficiency being about 80%, the actual annual ROI is about 150% or 2.5X (this does not account for initial equiptment costs). 
   
   Boiler Economic Efficiency - since the only boiler output is in the form of heated steam, therms is the only unit used and it remains consistent with original energy effiency. The difference here accounts for maintenence cost of the boiler so the economic efficiency will drop based on how much work was done over time. This is a great tool in measuring the trends of a boiler efficiency accross its service life and when performing an economic analysis to determine payback on installing a replacement boiler. These boilers are currently young and have a healthy efficiency rating.
   
   Generator economic efficiency - this variable just isolates the generator component and determines how efficient just the electricity producing component is. This component alone makes electricity 4.5 times cheaper than grid-based electricity. This proves that theoretical energy conversions from kWh to Therms does not suffice. Power plants can produce this electricity cheaply but line losses accross miles of transmission and transformer step up/down losses makes grid efficiency pale in comparison to the operating efficiency of hospital cogeneration. The reasons for the lower overall Cogeneration Economic Efficiency is cheifly from heat losses in the prime mover.

##### Limitations

This program does not account for the cost differences for inital building of the plant. Upon comparisons to their initial costs, the ballpark figures are very similar with cogen being somewhat more expensive when equalizing for their respective power outputs. This is a limitation that would change economic efficiency, but not by any marked amount. Payback analysis for cogeneration is done separately with a basis of comparing inital costs to "economic efficiency" overtime. This gives Economic Efficiency tremendous utility for economic analysis and is not harmed by discounting initial costs. 

Since the boilers and cogeneration plant are of different ages, their relative efficiencies might be slightly skewed. In an ideal scenario this analysis would compare two systems that were installed and used simultaneously. To combat this, datapoints were taken across a large timeframe that reflects the average annual usage of this equiptment. 

Constants vary greatly by source. Every constant used in this program provides a source but some fluctuate and are imperfect. For example kWh and natral gas prices are always changing and legislative intervention (interruptible/noninterruptible rates) are unpredictable when measuring accross time. 

No analysis is perfect and other limitations are the ones not yet realized. Any cogeneration or boiler plant nuances that I've missed could potentially skew these results. For those viewing this project, please add any improvements in a pull request or contact me with suggestions. 

## Conclusion

Cogeneration, especially in hospitals, is remarkably efficient and a useful tool for reducing emmisions. Local Laws 95 and 97 do not account for this because they isolate NYC buildings; an area outside of where most electricity generation occurs. Rapid electrification of cities may not be the best approach for reducing emmissions because electricity generation is still is heavily reliant on the burning of fossil fuels. As long as NYC relies on hydrocarbon-based power plants for electricity, grid nodes are a favorable option for emission reduction and should not be penalized by regulators. This project calls for more research into the requirements of Local Laws 95 and 97 as they pertain to hospitals and other energy prosumers. 